In [116]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import parse
import json
import copy

In [117]:
def parse_transcript(filename):
    spaces_regex = re.compile("^(\s*).*")
    location_regex = re.compile("^\s*(INT\.|EXT\.)")
    line_list = []
    options = [0] * 100
    opt = []
    transcript = []

    text_file = open(filename, "r")
    lines = text_file.readlines()
    for l in lines:
        li = l.strip(' \n\t\r')
        if li != "":
            spmatch = spaces_regex.search(l)
            spaces_number = len(spmatch.group(1))
            options[spaces_number] += 1
            line_list.append((li, spaces_number))
    for i in range(0, 50):
        if options[i] >= 20:
            opt.append(i)
    print(opt)
    text_file.close()

    speaker = ""
    utterance = ""

    for l in lines:
        li = l.strip(' \n\t\r')
        if li != "":
            spmatch = spaces_regex.search(l)
            spaces_number = len(spmatch.group(1))
            if spaces_number == opt[2]:
                if utterance != "" and speaker != "":
                    transcript.append(
                        {'speaker': speaker, 'utterance': utterance.strip()})
                    utterance = ""
                speaker = re.sub(r'\([^()]*\)', '', li).strip(' \n\t\r')
            elif spaces_number == opt[1]:
                utterance += " " + li
            else:
                if utterance != "" and speaker != "":
                    transcript.append(
                        {'speaker': speaker, 'utterance': utterance.strip()})
                    utterance = ""
                    speaker = ""

    return transcript

In [118]:
SCRIPT_PATH = "./data/scripts/Titanic.txt"
MOVIE_NAME = "Titanic"
parsed_transcript = (parse_transcript(SCRIPT_PATH))

[0, 28, 30, 31, 32, 33, 34, 35, 36]


In [119]:
def make_line_dict(transcript):
    line_dict = dict()

    print(parsed_transcript[0])
    
    for i in np.arange(len(parsed_transcript)):
        speaker = parsed_transcript[i]['speaker']
        line = parsed_transcript[i]['utterance']
        if speaker in line_dict.keys():
            line_dict[speaker] += [line]
        else:
            line_dict[speaker] = [line]
            
    return line_dict

In [120]:
credits = pd.read_csv('./data/tmdb_5000_credits.csv')

In [121]:
movie_titles = np.array(credits["title"].tolist())
movie_num = np.where(movie_titles == "Titanic")
movie_cast = np.array(credits["cast"].tolist())[movie_num][0]
line_dict = make_line_dict(parsed_transcript)

{'utterance': '(to the video crew)', 'speaker': 'a screenplay by'}


In [122]:
movie_char_list = json.loads(movie_cast)
gender_dict = dict()
movie_char_list_copy = copy.deepcopy(movie_char_list)

for speaker in line_dict.keys():
    speaker_dist = list()
    for d in movie_char_list_copy:
        if speaker.lower() == d['character'].lower() or speaker.lower() in ([x.lower() for x in d['character'].split(' ')]):            
            gender_dict[d['character']] = speaker, d['name'], d['gender']
            movie_char_list_copy.remove(d)
            break

In [126]:
ethnicities = pd.read_csv('./data/ethnicelebs.csv', header = None)
actor_names = ethnicities[0].tolist()
actor_ethnicities = ethnicities[1].tolist()

ethnicity_dict = dict()
            
for character in gender_dict.keys():
    speaker, name, gender = gender_dict[character]
    actor = '-'.join(name.lower().split(' '))
    try:
        actor_ethnicity = actor_ethnicities[actor_names.index(actor)]
        ethnicity_dict[character] = speaker, name, actor_ethnicity
    except ValueError:
        pass

gender_count = dict()
gender_chars = dict()
total_count = 0
total_chars = 0
for character in gender_dict.keys():
    speaker, name, gender = gender_dict[character]
    total_count += num_lines
    total_chars += 1
    num_lines = len(line_dict[speaker])
    if gender in gender_count.keys():
        gender_count[gender] += num_lines
        gender_chars[gender] += 1
    else:
        gender_count[gender] = num_lines
        gender_chars[gender] = 1
    
for g in gender_dict.keys():
    speaker, name, gender = gender_dict[g]
    num_lines = len(line_dict[speaker])
    print(speaker)
    print(num_lines / total_count)

LIZZY
0.058823529411764705
BODINE
0.058823529411764705
LOVETT
0.058823529411764705
REPORTER
0.058823529411764705
MASTER AT ARMS
0.058823529411764705
OLD ROSE
0.6470588235294118
STEWARD BARNES
0.058823529411764705


In [128]:
ethnicities_count = dict()

for character in ethnicity_dict.keys():
    speaker, name, ethnicity = ethnicity_dict[character]
    print(speaker)
    print(name)
    print(ethnicity)
    num_lines = len(line_dict[speaker])
    ethnicities = [x.strip() for x in ethnicity.split(",")]
    ethnicities_list = list()
    
    for ethnicity in ethnicities:
        e = ethnicity.split(' ')
        if len(e) == 0:
            ethnicities_list += [e[0]]
        else:
            np_e = np.array(e)
            e = []
            for e_num in np_e:
                if not any(char.isdigit() or char == "(" or char == ")" for char in e_num):
                    if  e_num[:1].upper() + e_num[1:] == e_num:
                        e += [e_num]
            ethnicities_list += [' '.join(e)]
    
    for ethnicity in ethnicities_list:
        if ethnicity in ethnicities_count.keys():
            ethnicities_count[ethnicity] += num_lines
        else:
            ethnicities_count[ethnicity] = num_lines
            
print(ethnicities_count)

LOVETT
Bill Paxton
English, Scots-Irish/Northern Irish, Scottish, Austrian, German, French, Swiss, Dutch, distant Welsh, remote Norwegian
{'Swiss': 1, 'English': 1, 'Welsh': 1, 'Norwegian': 1, 'Austrian': 1, 'Scottish': 1, 'Scots-Irish/Northern Irish': 1, 'Dutch': 1, 'French': 1, 'German': 1}
